In [1]:
import sys
sys.path.append('/Users/simon/git/burns/code/')
data_dir = '/Users/simon/Dropbox/BioResearch/Meta_clustering/Burns/ink_files/'

In [2]:
prefixes = ['BlBi','FoPe','Rhod','SiIB','SuIB','BlIG','HMIG']

In [5]:
import glob
files = []
for prefix in prefixes:
    files += glob.glob(data_dir + prefix + '*.mzML')
print files

['/Users/simon/Dropbox/BioResearch/Meta_clustering/Burns/ink_files/BlBia.mzML', '/Users/simon/Dropbox/BioResearch/Meta_clustering/Burns/ink_files/BlBib.mzML', '/Users/simon/Dropbox/BioResearch/Meta_clustering/Burns/ink_files/BlBic.mzML', '/Users/simon/Dropbox/BioResearch/Meta_clustering/Burns/ink_files/FoPea.mzML', '/Users/simon/Dropbox/BioResearch/Meta_clustering/Burns/ink_files/FoPeb.mzML', '/Users/simon/Dropbox/BioResearch/Meta_clustering/Burns/ink_files/FoPec.mzML', '/Users/simon/Dropbox/BioResearch/Meta_clustering/Burns/ink_files/Rhoda.mzML', '/Users/simon/Dropbox/BioResearch/Meta_clustering/Burns/ink_files/Rhodb.mzML', '/Users/simon/Dropbox/BioResearch/Meta_clustering/Burns/ink_files/Rhodc.mzML', '/Users/simon/Dropbox/BioResearch/Meta_clustering/Burns/ink_files/SiIBa.mzML', '/Users/simon/Dropbox/BioResearch/Meta_clustering/Burns/ink_files/SiIBb.mzML', '/Users/simon/Dropbox/BioResearch/Meta_clustering/Burns/ink_files/SiIBc.mzML', '/Users/simon/Dropbox/BioResearch/Meta_clustering/B

In [7]:
from load_di import load_di
data = {}
for f in files:
    data[f] = load_di(f)

In [8]:
normalised_data = {}
for f,spec in data.items():
    mz,inte = zip(*spec)
    max_inte = max(inte)
    new_inte = [1000.0*i/max_inte for i in inte]
    normalised_data[f] = zip(mz,new_inte)

In [10]:
from matching import Greedy
g = Greedy()
mp = g.process(normalised_data,5)

12401 peaks


In [12]:
short_names = {}
for s in normalised_data.keys():
    short_names[s] = s.split('/')[-1]
name_list = sorted(normalised_data.keys())

['/Users/simon/Dropbox/BioResearch/Meta_clustering/Burns/ink_files/BlBia.mzML', '/Users/simon/Dropbox/BioResearch/Meta_clustering/Burns/ink_files/BlBib.mzML', '/Users/simon/Dropbox/BioResearch/Meta_clustering/Burns/ink_files/BlBic.mzML', '/Users/simon/Dropbox/BioResearch/Meta_clustering/Burns/ink_files/BlIGa.mzML', '/Users/simon/Dropbox/BioResearch/Meta_clustering/Burns/ink_files/BlIGb.mzML', '/Users/simon/Dropbox/BioResearch/Meta_clustering/Burns/ink_files/BlIGc.mzML', '/Users/simon/Dropbox/BioResearch/Meta_clustering/Burns/ink_files/FoPea.mzML', '/Users/simon/Dropbox/BioResearch/Meta_clustering/Burns/ink_files/FoPeb.mzML', '/Users/simon/Dropbox/BioResearch/Meta_clustering/Burns/ink_files/FoPec.mzML', '/Users/simon/Dropbox/BioResearch/Meta_clustering/Burns/ink_files/HMIGa.mzML', '/Users/simon/Dropbox/BioResearch/Meta_clustering/Burns/ink_files/HMIGb.mzML', '/Users/simon/Dropbox/BioResearch/Meta_clustering/Burns/ink_files/HMIGc.mzML', '/Users/simon/Dropbox/BioResearch/Meta_clustering/B

In [23]:
rows = []
for ps in mp:
    row = [ps.mean_mz]
    for name in name_list:
        peaks = filter(lambda x: x[2] == name,ps.peaks)
        if len(peaks) == 0:
            row.append('nan')
        else:
            peaks = sorted(peaks,key = lambda x: x[1], reverse = True)
            row.append(peaks[0][1])
    rows.append(row)
    
rows = sorted(rows,key = lambda x: sum([1 for r in x if r == 'nan']))

In [24]:
import csv
with open('spreadsheets/inks/inks.csv','w') as f:
    writer = csv.writer(f)
    heads = ['mz'] + [short_names[s] for s in name_list]
    writer.writerow(heads)
    for row in rows:
        writer.writerow(row)